In [ ]:
!apt update
!apt install zip unzip -y
!apt update
!apt install p7zip-full p7zip-rar -y

In [ ]:
%cd /workspace/content/DiffSinger
!pip install -r requirements.txt

In [ ]:
%cd /workspace/content/MakeDiffSinger/variance-temp-solution
!pip install -r requirements.txt

In [ ]:
# !pip install pydub

In [1]:
all_shits_not_wav_n_lab = "/workspace/content/raw_data/diffsinger_db"
data_type = "lab + wav (NNSVS format)"

# [Inference_Acoustic_1] **Edit Config for Acoustic Inference**

In [1]:
#Edit Config for Acoustic
import re
import os
import yaml
import random #for the random test files lmaoz

%cd /workspace/content
# clear_output()
#@markdown <font size="-1.5"> The training type you want to do
config_type = "acoustic" # @param ["acoustic", "variance"]
vocoder_ckpt_path = "/workspace/content/DiffSinger/checkpoints/nsf_hifigan_44.1k_hop512_128bin_2024.02/model.ckpt"

config_cap = config_type.upper()

spk_name = [folder_name for folder_name in os.listdir(all_shits_not_wav_n_lab) if os.path.isdir(os.path.join(all_shits_not_wav_n_lab, folder_name))]
if ".ipynb_checkpoints" in spk_name:
    spk_name.remove(".ipynb_checkpoints")

# i used spk_name for something else cus i forgor now imma just copy and paste it
spk_names = [folder_name for folder_name in os.listdir(all_shits_not_wav_n_lab) if os.path.isdir(os.path.join(all_shits_not_wav_n_lab, folder_name))]
num_spk = len(spk_name)
raw_dir = []
for folder_name in spk_name:
    folder_path = os.path.join(all_shits_not_wav_n_lab, folder_name)
    raw_dir.append(folder_path)
if num_spk == 1:
    singer_type = "SINGLE-SPEAKER"
    diff_loss_type = "l2"
    f0_emb = "continuous"
    use_spk_id = False
    all_wav_files = []
    for root, dirs, files in os.walk("/workspace/content/raw_data/diffsinger_db"):
        for file in files:
            if file.endswith((".wav", ".ds")):
                full_path = os.path.join(root, file)
                all_wav_files.append(full_path)
    random.shuffle(all_wav_files)
    random_ass_wavs = all_wav_files[:3]
    random_ass_test_files = [os.path.splitext(os.path.basename(file))[0] for file in random_ass_wavs]

else:
    singer_type = "MULTI-SPEAKER"
    diff_loss_type = "l1"
    f0_emb = "discrete"
    use_spk_id = True
    folder_to_id = {folder_name: i for i, folder_name in enumerate(spk_name)}
    random_ass_test_files = []
    for folder_path in raw_dir:
        if data_type == "ds + wav (DiffSinger format)":
            audio_files = [f[:-4] for f in os.listdir(folder_path) if f.endswith(".ds")]
        else:
            audio_files = [f[:-4] for f in os.listdir(folder_path + "/wavs") if f.endswith(".wav")]
        folder_name = os.path.basename(folder_path)
        folder_id = folder_to_id.get(folder_name, -1)
        prefixed_audio_files = [f"{folder_id}:{audio_file}" for audio_file in audio_files]
        random_ass_test_files.extend(prefixed_audio_files[:3])
spk_id = []
for i, spk_name in enumerate(spk_name):
    spk_id_format = f"{i}:{spk_name}"
    spk_id.append(spk_id_format)

#@markdown <font size="-1.5"> Shallow Diffusion training for acoustic
use_shallow_diffusion = "true | gt_val" # @param ["false", "true | aux_val", "true | gt_val"]
if use_shallow_diffusion == "false":
    shallow = False
    gt_shallow = False
elif use_shallow_diffusion == "true | aux_val":
    shallow = True
    gt_shallow = False
else:
    shallow = True
    gt_shallow = True

#@markdown <font size="-1.5"> Half precision, or mixed precision can result in improved performance, achieving speedups on training (from [doc](https://lightning.ai/docs/pytorch/stable/common/trainer.html#precision))
# the reason why i dont add 64 is because colab is already dreadfully slow at 32 so yes im leaving it out
precision = "16-mixed" # @param ["32-true", "bf16-mixed", "16-mixed", "bf16", "16"]

#@markdown <font size="-1.5"> batch size setting, too low can cause bottleneck, too high can cause oom
batch_size = 12 # @param {type:"slider", min:1, max:100, step:1}


#@markdown <font size="-1.5"> Your model save path
save_dir = "/workspace/content/DiffSinger/MyModel_Acoustic" #@param {type:"string"}

binary_save_dir = save_dir + "/binary"

conf_dir = save_dir

#@markdown ....................................................................................................................................................................................................................................................................................................................................................................................................................................

#@markdown <font size="-1.5"> Option to use base model for finetuning

enable_finetuning = False # @param {type:"boolean"}


#@markdown <font size="-1.5"> Path to custom base model, leave blank to use [default](https://github.com/haru0l/diffsinger_models) models
#wtf haru i just looked at your readme"""""

base_model_path = "" # @param {type:"string"}

if enable_finetuning:
    pretrain = True
    if base_model_path:
        pretrain_ckpt = base_model_path
    else:
        pretrain_ckpt = f"/workspace/content/pretrain_models/{config_type}_pretrain.ckpt"
    finetune_strict_shapes = False
    finetune_ckpt_path = pretrain_ckpt
else:
    pretrain = False
    finetune_strict_shapes = True #default value
    finetune_ckpt_path = None #default value

#@markdown ....................................................................................................................................................................................................................................................................................................................................................................................................................................

#@markdown <font size="-1.5"> Model embeds check; Tension, Energy, Breathiness, Voicing | for both acoustic and variance
enable_embeds = True #@param {type: "boolean"}
if enable_embeds:
  tension_training = True
  energy_training = True
  breathiness_training = True
  voicing_training = True
else:
  tension_training = False
  energy_training = False
  breathiness_training = False
  voicing_training = False

pitch_training = "False"
if pitch_training == "False":
    pitch_training = False
    use_melody_encoder = False
    use_glide_embed = False
elif pitch_training == "True | Standard":
    pitch_training = True
    use_melody_encoder = False
    use_glide_embed = False
elif pitch_training == "True | Standard + Glide":
    pitch_training = True
    use_melody_encoder = False
    use_glide_embed = True
elif pitch_training == "True | MelodyEncoder":
    pitch_training = True
    use_melody_encoder = True
    use_glide_embed = False
else:
    pitch_training = True
    use_melody_encoder = True
    use_glide_embed = True

duration_training = True #@param {type: "boolean"}

#@markdown ....................................................................................................................................................................................................................................................................................................................................................................................................................................

#@markdown <font size="-1.5"> Pitch extractor algorithm

f0_ext = "rmvpe" # @param ["parselmouth", "rmvpe", "harvest"]
if f0_ext == "rmvpe":
    pe_ckpt_pth = "checkpoints/rmvpe/model.pt"
else:
    pe_ckpt_pth = None

#@markdown <font size="-1.5"> Select this if you want to use data augmentation (default pitch shift and time stretch values)
data_aug = True # Acoustic

with open("/workspace/content/DiffSinger/configs/base.yaml", "r") as config:
    mother = yaml.safe_load(config)
mother["pl_trainer_precision"] = precision
with open("/workspace/content/DiffSinger/configs/base.yaml", "w") as config:
    yaml.dump(mother, config)

if  data_type == "ds + wav (DiffSinger format)":
    prefer_ds = True
else:
    prefer_ds = False

if config_type == "acoustic":
    with open("/workspace/content/DiffSinger/configs/acoustic.yaml", "r") as config:
        bitch_ass_config = yaml.safe_load(config)
    bitch_ass_config["vocoder_ckpt"] = vocoder_ckpt_path
    bitch_ass_config["speakers"] = spk_names
    bitch_ass_config["test_prefixes"] = random_ass_test_files
    bitch_ass_config["raw_data_dir"] = raw_dir
    bitch_ass_config["num_spk"] = num_spk
    bitch_ass_config["use_spk_id"] = use_spk_id
    #bitch_ass_config["spk_ids"] = spk_id
    bitch_ass_config["diff_loss_type"] = diff_loss_type
    bitch_ass_config["f0_embed_type"] = f0_emb
    bitch_ass_config["binary_data_dir"] = binary_save_dir
    bitch_ass_config["dictionary"] = "dictionaries/custom_dict.txt"
    bitch_ass_config["augmentation_args"]["random_pitch_shifting"]["enabled"] = data_aug
    bitch_ass_config["augmentation_args"]["random_time_stretching"]["enabled"] = data_aug
    bitch_ass_config["use_key_shift_embed"] = data_aug
    bitch_ass_config["use_speed_embed"] = data_aug
    bitch_ass_config["max_batch_size"] = batch_size
    bitch_ass_config["pe"] = f0_ext
    bitch_ass_config["use_energy_embed"] = energy_training
    bitch_ass_config["use_breathiness_embed"] = breathiness_training
    bitch_ass_config["use_tension_embed"] = tension_training
    bitch_ass_config["use_voicing_embed"] = voicing_training

    bitch_ass_config["pe_ckpt"] = pe_ckpt_pth
    #shallow diff stuff
    bitch_ass_config["use_shallow_diffusion"] = shallow
    bitch_ass_config["shallow_diffusion_args"]["val_gt_start"] = gt_shallow
    #pretrain stuff
    bitch_ass_config["finetune_enabled"] = enable_finetuning
    bitch_ass_config["finetune_ckpt_path"] = finetune_ckpt_path
    bitch_ass_config["finetune_strict_shapes"] = finetune_strict_shapes

    with open("/workspace/content/DiffSinger/configs/acoustic.yaml", "w") as config:
        yaml.dump(bitch_ass_config, config)
else:
    with open("/workspace/content/DiffSinger/configs/variance.yaml", "r") as config:
        bitch_ass_config = yaml.safe_load(config)
    bitch_ass_config["speakers"] = spk_names
    bitch_ass_config["test_prefixes"] = random_ass_test_files
    bitch_ass_config["raw_data_dir"] = raw_dir
    bitch_ass_config["num_spk"] = num_spk
    bitch_ass_config["use_spk_id"] = use_spk_id
    bitch_ass_config["diff_loss_type"] = diff_loss_type
    bitch_ass_config["binary_data_dir"] = binary_save_dir
    bitch_ass_config["dictionary"] = "dictionaries/custom_dict.txt"
    bitch_ass_config["max_batch_size"] = batch_size
    bitch_ass_config["pe"] = f0_ext # i think variance uses it for pitch ref as ground-truth for pitch training soooo
    bitch_ass_config["pe_ckpt"] = pe_ckpt_pth #same goes to this one

    bitch_ass_config["predict_energy"] = energy_training
    bitch_ass_config["predict_breathiness"] = breathiness_training
    bitch_ass_config["predict_tension"] = tension_training
    bitch_ass_config["predict_pitch"] = pitch_training
    bitch_ass_config["predict_voicing"] = voicing_training

    bitch_ass_config["use_melody_encoder"] = use_melody_encoder
    bitch_ass_config["use_glide_embed"] = use_glide_embed
    bitch_ass_config["predict_dur"] = duration_training
    bitch_ass_config["binarization_args"]["prefer_ds"] = prefer_ds
    #pretrain stuff
    bitch_ass_config["finetune_enabled"] = enable_finetuning
    bitch_ass_config["finetune_ckpt_path"] = finetune_ckpt_path
    bitch_ass_config["finetune_strict_shapes"] = finetune_strict_shapes

    with open("/workspace/content/DiffSinger/configs/variance.yaml", "w") as config:
        yaml.dump(bitch_ass_config, config)

os.makedirs(save_dir, exist_ok=True)


with open("/workspace/content/DiffSinger/utils/hparams.py", "r") as f:
    hparams_py_read = f.read()
hparams_py_read = re.sub(r"args_work_dir\s*=\s*.*", f"args_work_dir = '{save_dir}'", hparams_py_read)
with open("/workspace/content/DiffSinger/utils/hparams.py", "w") as f:
    f.write(hparams_py_read)

with open("/workspace/content/DiffSinger/utils/training_utils.py", "r") as f:
    training_utils_stuff = f.read()
training_utils_stuff = re.sub("relative_path\s*=\s*.*", "relative_path = filepath.relative_to(Path('/workspace/content').resolve())", training_utils_stuff)
with open("/workspace/content/DiffSinger/utils/training_utils.py", "w") as f:
    f.write(training_utils_stuff)

print("config updated! see below for config's information")
print("|")
print("|")
print("|")
print(f"+++---{config_cap} {singer_type} TRAINING---+++")
print("|")
print("|")
print("|")
print("+++---user's settings---+++")
print("\n")
print(f"speaker name: {spk_names}")
print("\n")
print(f"data augmentation: {data_aug}")
print("\n")
print(f"pitch extractor: {f0_ext}")
print("\n")
print(f"binary data save directory: {binary_save_dir}")
print("\n")
print(f"your model will be saved to: {save_dir}")
print("\n")
print("==========================================================================================")
print("\n")
print("+++---other auto-defined settings---+++")
print("\n")
print(f"test files (auto selected): {random_ass_test_files}")
print("\n")
print("dictionary (auto generated): custom_dict.txt")
print("\n")
print(f"max_batch_size: {batch_size}")
print("\n")
print("==========================================================================================")
print("\n")
print("if you don't like or disagree with any of these options,")
print(f"you can go and edit the config at [/workspace/content/DiffSinger/configs/{config_type}.yaml]")


/workspace/content
config updated! see below for config's information
|
|
|
+++---ACOUSTIC SINGLE-SPEAKER TRAINING---+++
|
|
|
+++---user's settings---+++


speaker name: ['data']


data augmentation: True


pitch extractor: rmvpe


binary data save directory: /workspace/content/DiffSinger/MyModel_Acoustic/binary


your model will be saved to: /workspace/content/DiffSinger/MyModel_Acoustic




+++---other auto-defined settings---+++


test files (auto selected): ['tiger_024_seg001', 'tiger_014_seg006', 'tiger_020_seg002']


dictionary (auto generated): custom_dict.txt


max_batch_size: 12




if you don't like or disagree with any of these options,
you can go and edit the config at [/workspace/content/DiffSinger/configs/acoustic.yaml]


/usr/local/lib/python3.10/dist-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


# [Inference_Acoustic_2] **Acoustic Inference**

In [2]:

!python /workspace/content/DiffSinger/scripts/infer.py acoustic /workspace/inference_variance_result_long.ds --exp /workspace/content/DiffSinger/MyModel_Acoustic --out /workspace --title inference_acoustic_result_long

| found ckpt by name: /workspace/content/DiffSinger/MyModel_Acoustic
| Hparams chains:  []
| Hparams: 
K_step: 400, K_step_infer: 400, accumulate_grad_batches: 1, audio_num_mel_bins: 128, audio_sample_rate: 44100, 
augmentation_args: {'fixed_pitch_shifting': {'enabled': False, 'scale': 0.5, 'targets': [-5.0, 5.0]}, 'random_pitch_shifting': {'enabled': True, 'range': [-5.0, 5.0], 'scale': 0.75}, 'random_time_stretching': {'enabled': True, 'range': [0.5, 2.0], 'scale': 0.75}}, base_config: [], binarization_args: {'num_workers': 0, 'shuffle': True}, binarizer_cls: preprocessing.acoustic_binarizer.AcousticBinarizer, binary_data_dir: /workspace/content/DiffSinger/MyModel_Acoustic/binary, 
breathiness_smooth_width: 0.12, clip_grad_norm: 1, dataloader_prefetch_factor: 2, dataset_size_key: lengths, dictionary: dictionaries/custom_dict.txt, 
diff_accelerator: ddim, diff_decoder_type: wavenet, diff_loss_type: l2, diff_speedup: 10, dilation_cycle_length: 4, 
dropout: 0.1, ds_workers: 4, enc_ffn_k

In [28]:

# gender [-1,1]: [male, female]
!python /workspace/content/DiffSinger/scripts/infer.py acoustic /workspace/inference_variance_result.ds --exp /workspace/content/DiffSinger/MyModel_Acoustic --out /workspace --title inference_acoustic_depth --depth 7

| found ckpt by name: /workspace/content/DiffSinger/MyModel_Acoustic
| Hparams chains:  []
| Hparams: 
K_step: 400, K_step_infer: 400, accumulate_grad_batches: 1, audio_num_mel_bins: 128, audio_sample_rate: 44100, 
augmentation_args: {'fixed_pitch_shifting': {'enabled': False, 'scale': 0.5, 'targets': [-5.0, 5.0]}, 'random_pitch_shifting': {'enabled': True, 'range': [-5.0, 5.0], 'scale': 0.75}, 'random_time_stretching': {'enabled': True, 'range': [0.5, 2.0], 'scale': 0.75}}, base_config: [], binarization_args: {'num_workers': 0, 'shuffle': True}, binarizer_cls: preprocessing.acoustic_binarizer.AcousticBinarizer, binary_data_dir: /workspace/content/DiffSinger/MyModel_Acoustic/binary, 
breathiness_smooth_width: 0.12, clip_grad_norm: 1, dataloader_prefetch_factor: 2, dataset_size_key: lengths, dictionary: dictionaries/custom_dict.txt, 
diff_accelerator: ddim, diff_decoder_type: wavenet, diff_loss_type: l2, diff_speedup: 10, dilation_cycle_length: 4, 
dropout: 0.1, ds_workers: 4, enc_ffn_k